In [1]:
import selenium
from Scraping import *

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/Rgao/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [2]:
driver.get("http://www.randhawa.us/games/retailer/nyu.html")

# Define Strategy

In [56]:
# import gurobi
# from gurobi import *
# prob = Model('Optimization')
# x9 = prob.addVar(vtype=GRB.INTEGER,lb = 0, name = 'x_.9')
# x8= prob.addVar(lb = 0, name = 'x_.8')
# x6 = prob.addVar(lb = 0, name = 'x_.6')
# xM = prob.addVar(lb = 0, name = 'x_M')
# a=70
# p=60
# prob.addConstr(xM>=1,name="C1")
# prob.addConstr(a*(x9+x8+x6+xM)<= 2000,name="C2")
# prob.addConstr(x9+x8+x6+xM <= 15,name="C3")
# obj = a*x9*p*.9 + a*x8*p*.8 + a*x6*p*.6 + a*xM*p
# prob.setObjective(obj, GRB.MAXIMIZE)

import pulp as plp
import numpy as np
def optimizer(a):
    model = plp.LpProblem("Retail_Game", plp.LpMaximize)
    I = range(4)
    X = plp.LpVariable.dicts(name='X', indexs=(I),lowBound=0, cat=plp.LpInteger)
    discounted_price = [60,54,48,36]
    demand_lift = [1,1.31,1.73,2.81]
    model += plp.lpSum(a*discounted_price[i]*demand_lift[i]*X[i] for i in I)
    model += X[0] >= 1
    model += plp.lpSum(a*demand_lift[i]*X[i] for i in I) <= 2000
    model += plp.lpSum(X[i] for i in I) <= 15
    model.solve()
    print("="*30,"\nSolution Status:", plp.LpStatus[model.status])

    # Results
    obj = plp.value(model.objective)
    print("The result is: ${}".format(obj))
    values=[]
    print("Optimal Soultion:")
    for v in model.variables():
        print(v.name, "=", v.varValue)
        values.append(v.varValue)
    return(model,values)
model,choices=optimizer(100)

Solution Status: Optimal
The result is: $106578.0
Optimal Soultion:
X_0 = 3.0
X_1 = 9.0
X_2 = 3.0
X_3 = 0.0


In [57]:
all_choices=[]
for i in range(0,len(choices)):
    all_choices.extend([i]*int(choices[i]))

In [58]:
all_choices

[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]

In [18]:
table

,Week,Price,Sales,Remaining
0,1,60,96,1904
1,2,60,104,1800


In [65]:
def strategy(table,choices,chosenchoices):
    
    week=max(table['Week'].astype(int))
    if(week > 1):
        prev_choice=chosenchoices[week-1]
        lift=(int(table['Sales'][week-1])-int(table['Sales'][week-2]))/int(table['Sales'][week-2])
        lastchoice=max(chosenchoices)
        remainingweeks=15-week
        canfinish=False
        remaining=table['Remaining'][week-1]
#         if(remainingweeks*table['Sales'][week-1]>remaining):
#             canfinish=True
        
        if(lift<0):
            print("negative lift: "+ str(lift))
            return(min(lastchoice+1,3))
    return(0)

def tiancheng_strategy(table,choices,chosenchoices,already_calculated=None):
    week=max(table['Week'].astype(int))
    if(week == 1):
        model,values=optimizer(int(table['Sales'][week-1]))
        all_choices=[]
        for i in range(0,len(choices)):
            all_choices.extend([i]*int(values[i]))
        return(all_choices[week],all_choices)
    elif(already_calculated!=None):
        return(already_calculated[week],already_calculated)
    else:
        return(0,0)

#tiancheng_strategy(table,choices.chosenchoices,already_calculated)
#strategy(table,[0,1,2,3])
#table

### Get Buttons

In [31]:
html_source = driver.page_source

# Controls
restart_game = driver.find_element_by_id('practiceButton')
maintain = driver.find_element_by_id('maintainButton')
ten = driver.find_element_by_id('tenButton')
twenty = driver.find_element_by_id('twentyButton')
forty = driver.find_element_by_id('fortyButton')



def main_game_click(click_type=0):
#     global click_count
#     if click_count > 15:
#         print('Error: cannot go over 15 weeks.')
#         return
    if click_type == 0:
        maintain.click()
    if click_type == 1:
        ten.click()
    if click_type ==2:
        twenty.click()
    if click_type == 3:
        forty.click()
#     click_count += 1


def get_results(results_df, performance_df, game_id):
    table = driver.find_elements_by_id("result-table")
    for row in table:
        for cell_number in range(1, 16):
            cell = row.find_elements_by_tag_name("tr")[cell_number]
            print(cell.text)
            results_list = cell.text.split(' ')
            print(results_list)
            results_df = results_df.append({'Round': game_id, 'Week': results_list[0], 'Price': results_list[1], 'Sales': results_list[2],
                               'Remaining': results_list[3]}, ignore_index=True)

    revenue = driver.find_element_by_id("rev")
    perfect = driver.find_element_by_id("perfect")
    difference = driver.find_element_by_id("percentage")
    print(f'Your revenue is: {revenue.text}')
    print(f'Perfect revenue is: {perfect.text}')
    print(f'Difference is: {difference.text}')
    performance_df = performance_df.append({'Round': game_id, 'Revenue': revenue.text, 'Perfect': perfect.text, 'Difference': difference.text.split(' ')[2]}, ignore_index=True)

    return results_df, performance_df

# Try out controls
'''
main_game_click()
main_game_click()
main_game_click()
main_game_click()
main_game_click('ten')
main_game_click()
main_game_click()
main_game_click()
main_game_click('twenty')
main_game_click()
main_game_click()
main_game_click()
main_game_click('forty')
'''

results_df = pd.DataFrame(columns=['Round', 'Week', 'Price', 'Sales', 'Remaining'])
performance_df = pd.DataFrame(columns=['Round', 'Revenue', 'Perfect', 'Difference'])


In [34]:
all_data=[]
foresights=[]
driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
#choices=[[0]*15,[1]+[0]*14,[2]+[0]*14,[3]+[0]*14] # These are the strategies to scrape
#choices=[choices[0]]
attempts=10

for attempt in range(attempts):#choices:
    chosenchoices=[0]
    driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
    for i in range(0,15):
        #hoice=choices[random.randint(0,len(choices)-1)]

        data=driver.find_elements_by_xpath("//table[contains(@id,'result-table')]")[0].text
        splitdata=data.split("\n")
        columns=splitdata[0].split(" ")
        values_split=[]
        for values in splitdata[1:len(splitdata)]:
            values_split.append(values.split(" "))

        table=pd.DataFrame(values_split)
        table.columns=columns[0:4]
        
        choice=strategy(table,[0,1,2,3],chosenchoices)#choicerange[i]
        chosenchoices.append(choice)
        main_game_click(choice)

        if(choice==3):
            data=driver.find_elements_by_xpath("//table[contains(@id,'result-table')]")[0].text
            splitdata=data.split("\n")
            columns=splitdata[0].split(" ")
            values_split=[]
            for values in splitdata[1:len(splitdata)]:
                values_split.append(values.split(" "))

            table=pd.DataFrame(values_split)
            table.columns=columns[0:4]
            chosenchoices.extend([0]*(13-i))
            break
    foresight=driver.find_element_by_xpath("//div[contains(@style,'position:relative;top:10px;left:6px;')]").text
    table['foresight']=foresight
    table['Choice']=chosenchoices
    table['diff']=float(table['foresight'][0].split(": ")[3].replace("%",""))
    table['revenue']=float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$","").replace(",",""))
    table['perfect']=float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$","").replace(",",""))
    table['Attempt']=attempt
    all_data.append(table)
# print(table)

negative lift: -0.1038961038961039
negative lift: -0.16521739130434782
negative lift: -0.16
negative lift: -0.9459459459459459
negative lift: -0.04201680672268908
negative lift: -0.12280701754385964
negative lift: -0.46564885496183206
negative lift: -0.37267080745341613
negative lift: -0.1152073732718894
negative lift: -0.7479674796747967
negative lift: -0.2638888888888889
negative lift: -0.2328767123287671
negative lift: -0.0962566844919786
negative lift: -0.005917159763313609
negative lift: -0.40476190476190477
negative lift: -0.13414634146341464
negative lift: -0.2482758620689655
negative lift: -0.23225806451612904
negative lift: -0.3146067415730337
negative lift: -0.7295081967213115
negative lift: -0.2236842105263158
negative lift: -0.1511627906976744
negative lift: -0.0273972602739726
negative lift: -0.47096774193548385
negative lift: -0.13333333333333333
negative lift: -0.18681318681318682
negative lift: -0.5195530726256983
negative lift: -0.06382978723404255
negative lift: -0.26

In [36]:
table

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Attempt
0,1,60,93,1907,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
1,2,60,94,1813,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
2,3,60,88,1725,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
3,4,54,163,1562,"Your revenue: $88,104, Perfect foresight strat...",1,15.6,88104.0,104388.0,9
4,5,54,119,1443,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
5,6,48,206,1237,"Your revenue: $88,104, Perfect foresight strat...",2,15.6,88104.0,104388.0,9
6,7,48,163,1074,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
7,8,36,331,743,"Your revenue: $88,104, Perfect foresight strat...",3,15.6,88104.0,104388.0,9
8,9,36,364,379,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
9,10,36,208,171,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9


In [37]:
all_data=pd.concat(all_data).reset_index(drop=True)
all_data

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect,Attempt
0,1,60,72,1928,"Your revenue: $81,846, Perfect foresight strat...",0,4.6,81846.0,85752.0,0
1,2,60,77,1851,"Your revenue: $81,846, Perfect foresight strat...",0,4.6,81846.0,85752.0,0
2,3,60,69,1782,"Your revenue: $81,846, Perfect foresight strat...",0,4.6,81846.0,85752.0,0
3,4,54,115,1667,"Your revenue: $81,846, Perfect foresight strat...",1,4.6,81846.0,85752.0,0
4,5,54,96,1571,"Your revenue: $81,846, Perfect foresight strat...",0,4.6,81846.0,85752.0,0
...,...,...,...,...,...,...,...,...,...,...
145,11,36,142,29,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
146,12,36,29,0,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
147,13,36,0,0,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9
148,14,36,0,0,"Your revenue: $88,104, Perfect foresight strat...",0,15.6,88104.0,104388.0,9


In [203]:
pd.concat(all_data).to_csv("games.csv")

In [4]:
games=pd.read_csv("games.csv")

In [5]:
games

,Unnamed: 0,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
0,0,1,60,83,1917,"Your revenue: $84,942, Perfect foresight strat...",0,13.8,84942,98562
1,1,2,60,110,1807,"Your revenue: $84,942, Perfect foresight strat...",0,13.8,84942,98562
2,2,3,60,91,1716,"Your revenue: $84,942, Perfect foresight strat...",0,13.8,84942,98562
3,3,4,54,75,1641,"Your revenue: $84,942, Perfect foresight strat...",1,13.8,84942,98562
4,4,5,48,100,1541,"Your revenue: $84,942, Perfect foresight strat...",2,13.8,84942,98562
...,...,...,...,...,...,...,...,...,...,...
2215,10,11,36,194,557,"Your revenue: $85,392, Perfect foresight strat...",0,9.4,85392,94260
2216,11,12,36,265,292,"Your revenue: $85,392, Perfect foresight strat...",0,9.4,85392,94260
2217,12,13,36,150,142,"Your revenue: $85,392, Perfect foresight strat...",0,9.4,85392,94260
2218,13,14,36,142,0,"Your revenue: $85,392, Perfect foresight strat...",0,9.4,85392,94260


# Tiancheng Strategy

In [45]:
already_calculated

[1, 2, 2, 3, 3, 3]

In [60]:
table

,Week,Price,Sales,Remaining,foresight
0,1,60,31,1969,"Your revenue: $5,028"
1,2,48,66,1903,"Your revenue: $5,028"


In [61]:
tiancheng_strategy(table,[0,1,2,3],chosenchoices,already_calculated=None)

(0, 0)

In [62]:
already_calculated

[1, 2, 2, 3, 3, 3]

In [67]:
table

,Week,Price,Sales,Remaining,foresight
0,1,60,119,1881,"Your revenue: $114,978"
1,2,60,111,1770,"Your revenue: $114,978"
2,3,60,171,1599,"Your revenue: $114,978"
3,4,60,152,1447,"Your revenue: $114,978"
4,5,60,53,1394,"Your revenue: $114,978"
5,6,60,149,1245,"Your revenue: $114,978"
6,7,60,170,1075,"Your revenue: $114,978"
7,8,60,101,974,"Your revenue: $114,978"
8,9,60,137,837,"Your revenue: $114,978"
9,10,60,141,696,"Your revenue: $114,978"


In [69]:
len(all_choices)

15

In [66]:
all_data=[]
foresights=[]
driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
#choices=[[0]*15,[1]+[0]*14,[2]+[0]*14,[3]+[0]*14] # These are the strategies to scrape
#choices=[choices[0]]
attempts=10

for attempt in range(attempts):#choices:
    chosenchoices=[0]
    driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
    for i in range(0,15):
        #hoice=choices[random.randint(0,len(choices)-1)]

        data=driver.find_elements_by_xpath("//table[contains(@id,'result-table')]")[0].text
        splitdata=data.split("\n")
        columns=splitdata[0].split(" ")
        values_split=[]
        for values in splitdata[1:len(splitdata)]:
            values_split.append(values.split(" "))

        table=pd.DataFrame(values_split)
        table.columns=columns[0:4]
        if(i==0):
            choice,already_calculated=tiancheng_strategy(table,[0,1,2,3],chosenchoices,already_calculated=None)#strategy(table,[0,1,2,3],chosenchoices)#choicerange[i]
        else:
            choice,already_calculated=tiancheng_strategy(table,[0,1,2,3],chosenchoices,already_calculated=already_calculated)
        print(i,choice)
        chosenchoices.append(choice)
        main_game_click(choice)

        if(choice==3):
            print("BREAK")
            data=driver.find_elements_by_xpath("//table[contains(@id,'result-table')]")[0].text
            splitdata=data.split("\n")
            columns=splitdata[0].split(" ")
            values_split=[]
            for values in splitdata[1:len(splitdata)]:
                values_split.append(values.split(" "))

            table=pd.DataFrame(values_split)
            table.columns=columns[0:4]
            chosenchoices.extend([0]*(13-i))
            break
    foresight=driver.find_element_by_xpath("//div[contains(@style,'position:relative;top:10px;left:6px;')]").text
    table['foresight']=foresight
    table['Choice']=chosenchoices
    table['diff']=float(table['foresight'][0].split(": ")[3].replace("%",""))
    table['revenue']=float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$","").replace(",",""))
    table['perfect']=float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$","").replace(",",""))
    table['Attempt']=attempt
    all_data.append(table)
# print(table)

Solution Status: Optimal
The result is: $73736.82
Optimal Soultion:
X_0 = 1.0
X_1 = 1.0
X_2 = 0.0
X_3 = 13.0
0 1
1 3
BREAK
Solution Status: Optimal
The result is: $113861.58
Optimal Soultion:
X_0 = 12.0
X_1 = 1.0
X_2 = 2.0
X_3 = 0.0
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 1
12 2
13 2
14 2


ValueError: Length of values (16) does not match length of index (14)